## Import Libraries

In [1]:
import pandas as pd
import requests
import urllib.parse
import urllib.request
import json
from tqdm import tqdm
import time

## Import Dataset

In [2]:
df_source = pd.read_csv('./data/safegraph-stores-geocode-out-manual.csv')

## Create Template for New Dataset

First, prepare the data by creating a dictionary of all cities. Once this dictionary is made convert it into a list of lists.

In [31]:
geocode_dict = {}
geocodes = []

# make dictionary of all cities
for i in range(len(df_source)):
    curr_geocode = str(df_source.loc[i, "geocode"])
    curr_geocode = curr_geocode[1:-1]
    geocode_dict[curr_geocode] = [df_source.loc[i, "address"], df_source.loc[i, "city"], 
                                  str(df_source.loc[i, "zip_code"]), str(df_source.loc[i, "store_skey"])]

# make list of lists to pass as input to make dataframe
for key in geocode_dict:
    geocodes += [key]
print('Number of geocodes:', len(geocode_dict))

Number of geocodes: 1026


## Call API for all the cities

In [36]:
for geocode in tqdm(geocodes):
    try:
        url = 'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/' + urllib.parse.quote_plus(str(geocode)) + '/2018-09-03/2022-05-29?unitGroup=metric&include=days&key=3FWLMZ9MF7M7QBH73WCBXT2QF&contentType=csv'
        req = urllib.request.Request(url)
        res = urllib.request.urlopen(req)
        
        with open('./data/historical_weather.csv',"ab") as f:
            f.write(res.read())
    except:
        print(geocode, 'could not be found in API.')

'\nfor geocode in tqdm(geocodes):\n    try:\n        url = \'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/\' + urllib.parse.quote_plus(str(geocode)) + \'/2018-09-03/2022-05-29?unitGroup=metric&include=days&key=3FWLMZ9MF7M7QBH73WCBXT2QF&contentType=csv\'\n        req = urllib.request.Request(url)\n        res = urllib.request.urlopen(req)\n        \n        with open(\'./data/historical_weather.csv\',"ab") as f:\n            f.write(res.read())\n    except:\n        print(geocode, \'could not be found in API.\')\n'

In [32]:
df = pd.read_csv('./data/historical_weather.csv')
df = df[df.name != 'name']
df.to_csv('./data/historical_weather_temp.csv', index=False)
df = pd.read_csv('./data/historical_weather_temp.csv')

In [33]:
for i in tqdm(range(len(df))):
    try:
        df.loc[i, 'address'] = geocode_dict[df.loc[i, 'name']][0]
        df.loc[i, 'city'] = geocode_dict[df.loc[i, 'name']][1]
        df.loc[i, 'zip_code'] = geocode_dict[df.loc[i, 'name']][2]
        df.loc[i, 'store_skey'] = geocode_dict[df.loc[i, 'name']][3]
    except:
        print('Error with', df.loc[i, 'name'])

100%|████████████████████████████████| 1399125/1399125 [04:05<00:00, 5692.99it/s]


In [34]:
df.reindex(columns=['store_skey', 'address', 'city', 'zip_code', 'name', 'datetime', 
                    'tempmax', 'tempmin', 'temp', 'feelslikemax',
                    'feelslikemin', 'feelslike', 'dew', 'humidity', 'precip', 'precipprob',
                    'precipcover', 'preciptype', 'snow', 'snowdepth', 'windgust',
                    'windspeed', 'winddir', 'sealevelpressure', 'cloudcover', 'visibility',
                    'solarradiation', 'solarenergy', 'uvindex', 'severerisk', 'sunrise',
                    'sunset', 'moonphase', 'conditions', 'description', 'icon', 'stations'])

,store_skey,address,city,zip_code,name,datetime,tempmax,tempmin,temp,feelslikemax,...,solarenergy,uvindex,severerisk,sunrise,sunset,moonphase,conditions,description,icon,stations
0,1061,2932 W. Norvell Bryant Highway,Lecanto,34461,"28.8919567,-82.4836527",2018-09-03,31.0,23.0,24.5,34.9,...,12.9,8,NaN,2018-09-03T07:08:47,2018-09-03T19:49:16,0.78,"Rain, Partially cloudy",Partly cloudy throughout the day with a chance...,rain,"72473700377,KOCF,72065500235,KCGC,72201412818,..."
1,1061,2932 W. Norvell Bryant Highway,Lecanto,34461,"28.8919567,-82.4836527",2018-09-04,30.8,23.0,26.2,36.7,...,14.5,8,NaN,2018-09-04T07:09:17,2018-09-04T19:48:06,0.83,"Rain, Partially cloudy",Partly cloudy throughout the day with a chance...,rain,"72473700377,KOCF,72065500235,KCGC,72201412818,..."
2,1061,2932 W. Norvell Bryant Highway,Lecanto,34461,"28.8919567,-82.4836527",2018-09-05,33.0,24.0,27.0,38.3,...,19.3,9,NaN,2018-09-05T07:09:47,2018-09-05T19:46:56,0.88,"Rain, Partially cloudy",Partly cloudy throughout the day with late aft...,rain,"72473700377,KOCF,72065500235,KCGC,72201412818,..."
3,1061,2932 W. Norvell Bryant Highway,Lecanto,34461,"28.8919567,-82.4836527",2018-09-06,32.0,23.0,26.1,37.0,...,17.6,8,NaN,2018-09-06T07:10:17,2018-09-06T19:45:45,0.93,"Rain, Partially cloudy",Partly cloudy throughout the day with rain.,rain,"72473700377,KOCF,72065500235,KCGC,72201412818,..."
4,1061,2932 W. Norvell Bryant Highway,Lecanto,34461,"28.8919567,-82.4836527",2018-09-07,32.0,22.1,26.1,35.7,...,21.2,9,NaN,2018-09-07T07:10:47,2018-09-07T19:44:34,0.97,Rain,Clear conditions throughout the day with rain.,rain,"72473700377,KOCF,72065500235,KCGC,72201412818,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1399120,1195,1465 Clements Bridge Rd.,Deptford,8096,"39.8295251,-75.1087912",2022-05-25,24.3,12.8,17.9,24.3,...,13.0,6,10.0,2022-05-25T05:37:54,2022-05-25T20:17:25,0.89,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"KVAY,F6990,72408013739,72408594732,72407493780..."
1399121,1195,1465 Clements Bridge Rd.,Deptford,8096,"39.8295251,-75.1087912",2022-05-26,23.0,11.9,18.2,23.0,...,11.9,9,10.0,2022-05-26T05:37:18,2022-05-26T20:18:14,0.93,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"KVAY,F6990,72408013739,72408594732,72407493780..."
1399122,1195,1465 Clements Bridge Rd.,Deptford,8096,"39.8295251,-75.1087912",2022-05-27,26.7,19.2,21.1,28.1,...,4.9,3,30.0,2022-05-27T05:36:43,2022-05-27T20:19:02,0.96,"Rain, Overcast",Cloudy skies throughout the day with rain.,rain,"KVAY,F5255,F6990,72408013739,72408594732,72407..."
1399123,1195,1465 Clements Bridge Rd.,Deptford,8096,"39.8295251,-75.1087912",2022-05-28,24.9,19.3,22.0,24.9,...,16.2,9,30.0,2022-05-28T05:36:10,2022-05-28T20:19:49,0.99,"Rain, Partially cloudy",Partly cloudy throughout the day with rain.,rain,"KVAY,F6990,72408013739,72408594732,72407493780..."


In [35]:
df.to_csv('./data/historical_weather_skeys.csv', index=False)